In [92]:
import json
import hashlib
from datetime import datetime
from breeze_connect import BreezeConnect
import urllib
import pandas as pd
import numpy as np
import math


In [3]:
Key = '73=63fT44781tY4C42L0i3491201v23A'
Secret = '11331XF5fm6c5&Y3K2Q99lp28347480+'
api_session = '36820063'

In [4]:


# Initialize SDK
breeze = BreezeConnect(api_key=Key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.

#print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus("your_api_key"))

# Generate Session
breeze.generate_session(api_secret=Secret,
                        session_token=api_session)

# Generate ISO8601 Date/DateTime String
import datetime
iso_date_string = datetime.datetime.strptime("07/12/2023","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
iso_date_time_string = datetime.datetime.strptime("07/12/2023 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'

In [5]:
#get customer details
breeze.get_customer_details(api_session=api_session) 

{'Success': {'exg_trade_date': {'NSE': '12-Mar-2024',
   'BSE': '12-Mar-2024',
   'FNO': '12-Mar-2024',
   'NDX': '12-Mar-2024'},
  'exg_status': {'NSE': 'X', 'BSE': 'X', 'FNO': 'Y', 'NDX': 'X'},
  'segments_allowed': {'Trading': 'Y',
   'Equity': 'Y',
   'Derivatives': 'D',
   'Currency': 'D'},
  'idirect_userid': 'AG911462',
  'idirect_user_name': 'ANKIT',
  'idirect_ORD_TYP': '',
  'idirect_lastlogin_time': '12-Mar-2024 17:13:42',
  'mf_holding_mode_popup_flg': 'N',
  'commodity_exchange_status': 'N',
  'commodity_trade_date': '12-Mar-2024',
  'commodity_allowed': 'X'},
 'Status': 200,
 'Error': None}

In [6]:
#get holdings
breeze.get_demat_holdings()

{'Success': [{'stock_code': 'SOUBAN',
   'stock_ISIN': 'INE683A01023',
   'quantity': '10',
   'demat_total_bulk_quantity': '10',
   'demat_avail_quantity': '0',
   'blocked_quantity': '0',
   'demat_allocated_quantity': '10'},
  {'stock_code': 'SOURE1',
   'stock_ISIN': 'INE683A20015',
   'quantity': '2',
   'demat_total_bulk_quantity': '0',
   'demat_avail_quantity': '2',
   'blocked_quantity': '0',
   'demat_allocated_quantity': '0'}],
 'Status': 200,
 'Error': None}

In [56]:
#get_stock details 
def get_histroy_data_stock(stock_code,from_date,to_date,interval='1day',exchange='NSE'):
    data = breeze.get_historical_data(interval=interval,
                                from_date= f"{from_date}T07:00:00.000Z",
                                to_date= f"{to_date}T07:00:00.000Z",
                                stock_code=stock_code,
                                exchange_code=exchange,
                                product_type="cash")
    if data['Status']==200:
        df = pd.DataFrame(data['Success'])[['datetime','stock_code','open','close','high','low']]
        for x in df.columns:
            try:
                df[x]=df[x].astype(float)
                print(x,'complete')
            except Exception as e:
                print('couldnt perform on',x)
                print('Error',e)
        return df
    else:
        print('got error')
        return data
    
def get_log_return(df):
    df['Log Returns'] = np.log(df['close'] / df['close'].shift(1))
    return df

In [57]:
data_1year = get_histroy_data_stock('CNXBAN','2023-02-12','2024-03-12')
data_30days = get_histroy_data_stock('CNXBAN','2024-02-12','2024-03-12')

couldnt perform on datetime
Error could not convert string to float: '2023-02-13 12:02:00'
couldnt perform on stock_code
Error could not convert string to float: 'CNXBAN'
open complete
close complete
high complete
low complete
couldnt perform on datetime
Error could not convert string to float: '2024-02-12 12:02:00'
couldnt perform on stock_code
Error could not convert string to float: 'CNXBAN'
open complete
close complete
high complete
low complete


In [71]:

data_1year = get_log_return(data_1year)
data_30days = get_log_return(data_30days)



In [72]:
print(30,data_30days['Log Returns'].mean())
print('1 year',data_1year['Log Returns'].mean())

30 0.0026528261752270478
1 year 0.0005137876046064782


In [106]:
def range_for_days(df,days):
    original = df
    last_price = original['close'].iloc[-1]
    df = df['Log Returns']
    mean = df.mean()
    sd = df.std()
    print('Calculated mean',mean,'\n')
    print('calculated sd', sd,'\n')
    average = days*mean
    std = days**(0.5)*sd
    print('adjusted average for',days,'is',average*100,'%\n')
    print('adjusted std for',days,'is',std*100,'%\n')

    ranges = {}
    for x,y in zip(range(1,4),[68,95,99.7]):
        lower_range = average-std*x
        upper_range = average+std*x
        lower_range_price = last_price*math.exp(lower_range)
        upper_range_price = last_price*math.exp(upper_range)

        
        print(f'Range for {x}sd {y}% confidence level is {lower_range} -- {upper_range}')
        print(f'Range in price for {x}sd {y}% confidence level is {lower_range_price} -- {upper_range_price}\n')
        ranges[y] = [lower_range_price,upper_range_price]
    
    return ranges

range_for_days(data_1year,30)


Calculated mean 0.0005137876046064782 

calculated sd 0.008378233122672945 

adjusted average for 30 is 1.5413628138194346 %

adjusted std for 30 is 4.58894727332492 %

Range for 1sd 68% confidence level is -0.030475844595054855 -- 0.06130310087144354
Range in price for 1sd 68% confidence level is 45907.25072965046 -- 50319.97009138237

Range for 2sd 95% confidence level is -0.07636531732830405 -- 0.10719257360469274
Range in price for 2sd 95% confidence level is 43848.196969168464 -- 52682.929820760895

Range for 3sd 99.7% confidence level is -0.12225479006155325 -- 0.15308204633794195
Range in price for 3sd 99.7% confidence level is 41881.4968635268 -- 55156.851036653126



{68: [45907.25072965046, 50319.97009138237],
 95: [43848.196969168464, 52682.929820760895],
 99.7: [41881.4968635268, 55156.851036653126]}

In [109]:
range_for_days(data_1year,8)

Calculated mean 0.0005137876046064782 

calculated sd 0.008378233122672945 

adjusted average for 8 is 0.41103008368518257 %

adjusted std for 8 is 2.3697221821615133 %

Range for 1sd 68% confidence level is -0.01958692098476331 -- 0.02780752265846696
Range in price for 1sd 68% confidence level is 46409.86276185661 -- 48662.3893841348

Range for 2sd 95% confidence level is -0.04328414280637844 -- 0.0515047444800821
Range in price for 2sd 95% confidence level is 45323.0065463122 -- 49829.32477513421

Range for 3sd 99.7% confidence level is -0.06698136462799359 -- 0.07520196630169723
Range in price for 3sd 99.7% confidence level is 44261.60303333941 -- 51024.243547632155



{68: [46409.86276185661, 48662.3893841348],
 95: [45323.0065463122, 49829.32477513421],
 99.7: [44261.60303333941, 51024.243547632155]}

In [86]:
print(a,sd)

0.1008 2.63592


In [97]:
8337*math.exp(-0.0695)

7777.254932027043

In [13]:
#get bank details
breeze.get_funds()

{'Success': {'bank_account': '3223108004024',
  'total_bank_balance': 0.0,
  'allocated_equity': 0.0,
  'allocated_fno': 0.0,
  'block_by_trade_equity': 0.0,
  'block_by_trade_fno': 0.0,
  'block_by_trade_balance': 0.0,
  'unallocated_balance': '0'},
 'Status': 200,
 'Error': None}